In [1]:
from six.moves import cPickle as pickle

import sys
# reload(sys)
# sys.setdefaultencoding('utf-8')

import os as os
import fnmatch
import argparse
import pathlib
import time
import itertools

import numpy as np
if "DISPLAY" not in os.environ:
    mpl.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import scipy.io as sio
import glob 

from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold  # k-fold cross validation StratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler  # data normalization
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier  # random forest model
from sklearn.pipeline import Pipeline # model pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import make_scorer


%reload_ext autoreload

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.rc('text', usetex=True)
plt.rc('font', family='times')

plt.rcParams.update({'font.size': 22})
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2



In [2]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [3]:
knownNames = glob.glob("../dataset/Dados_SONAR/*.mat")

X = [sio.loadmat(x)['ent_norm'] for x in knownNames]
Y = [(x.split('/')[-1][0]) for x in knownNames]

print(Y)

# Y = ['A' if y!='B' else y for y in Y]
# X2 = [sio.loadmat(x)['ent_norm'] for x in unknownNames]
# print(X)

minLengthVector = [np.shape(x)[0] for x in X]
minLengthKnown = int(np.min(minLengthVector))

# minLengthVector = [np.shape(x)[0] for x in X2]
# minLengthUnknown = int(np.min(minLengthVector))

minLength = minLengthKnown

le = LabelEncoder()
target = le.fit_transform(Y)

print(target)



# minLength = np.zeros(len(names))

# for i in range(len(names)):
#     x = sio.loadmat(str(names[i]))
#     y = x["ent_norm"]
#     minLength[i] = np.shape(y)[0]
# print(np.min(minLength))

['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'G', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'A', 'B', 'C', 'C', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'G', 'G', 'H', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'G', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'E', 'E', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'D',

In [4]:
def increase_samples(data, wLength, ovSamples, label=None):
    
    y = []
    newLabel = []
    for counter, x in enumerate(data):
        size = np.shape(x)[0]
        beginWindowIndex = 0
        endWindowIndex = wLength
        if label is None:
            label2 = []
        else:    
            label2 = label[counter]
        
        while True:
            vector = np.reshape(x[beginWindowIndex:endWindowIndex,:], -1)
            y.append(vector)
            beginWindowIndex = beginWindowIndex + ovSamples
            endWindowIndex = endWindowIndex + ovSamples
            newLabel.append(label2)
            if endWindowIndex > size:
                break
            
    y = np.stack(y)
    return y, newLabel 

In [5]:
windowLength = 10
# xMatrixUnknown = np.stack([x[:minLength,:].reshape(-1) for x in X2])
# xMatrix2 = np.stack([x[:minLength,:].reshape(-1) for x in X2])


# print(xMatrix.shape)

# print(target.shape)
# # xMatrix = np.stack([x.reshape(-1) for x in X])


# # X_train, X_test, y_train, y_test = train_test_split(xMatrix, target, test_size=0.3, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=42)

# print(X_train[0].shape)

# teste,label2 = increase_samples(X_train, 10, 1, y_train)


# # pca = PCA(10)
# # X_train = pca.fit_transform(X_train)
# # X_test = pca.fit_transform(X_test)

# model = Pipeline([('pca', PCA()), ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))])
# # model = Pipeline([ ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))])

# # model = Pipeline([('pca', PCA()), ('svm', svm.LinearSVC())])


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=42)


In [7]:
xMatrixTrainExtended,y_trainExtended = increase_samples(X_train, windowLength, 1, y_train)


In [8]:
model = Pipeline([('pca', PCA()), ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))])

In [9]:
xMatrixTest = np.stack([x[:windowLength,:].reshape(-1) for x in X_test])


In [10]:
fold = StratifiedKFold(10)


In [11]:
# parameters = {'pca__n_components': [252], 'rf__n_estimators': [82], 'rf__n_jobs': [-1],
#               'rf__min_samples_leaf': [35], 'rf__criterion': ['gini']}
metric = metrics.make_scorer(metrics.accuracy_score)
# metric = metrics.make_scorer(metrics.precision_score, average='weighted')



parameters = {'pca__n_components': sp_randint(10, 500), 'rf__n_estimators': sp_randint(10, 100), 'rf__n_jobs': [-1],
              'rf__min_samples_leaf': sp_randint(1, 100), 'rf__criterion': ['gini']}


# parameters = {'pca__n_components': sp_randint(10, 200)}

gridsearchcv = RandomizedSearchCV(model, parameters, scoring=metric, cv=fold, n_iter=10)

# print(xMatrixTrainExtended.shape)

# gridsearchcv = GridSearchCV(model, parameters, n_jobs=3, cv=fold, scoring=metric)
gridsearchcv.fit(xMatrixTrainExtended, y_trainExtended)

KeyboardInterrupt: 

In [ ]:
gridsearchcv.cv_results_

In [ ]:
# best_n_components = gridsearchcv.best_params_['pca__n_components']

xMatrixTestExtended,y_testExtended = increase_samples(X_test, windowLength, 1, y_test)

print(gridsearchcv.best_params_)
means = gridsearchcv.cv_results_['mean_test_score']
stds = gridsearchcv.cv_results_['std_test_score']

print(means,stds)
acc = metrics.accuracy_score(y_testExtended, gridsearchcv.best_estimator_.predict(xMatrixTestExtended))
F1 = metrics.f1_score(y_testExtended, gridsearchcv.best_estimator_.predict(xMatrixTestExtended), average='weighted')
prec = metrics.precision_score(y_testExtended, gridsearchcv.best_estimator_.predict(xMatrixTestExtended), average='weighted')
print(acc,F1,prec)

confusionMatrix = metrics.confusion_matrix(y_testExtended, gridsearchcv.best_estimator_.predict(xMatrixTestExtended))

plt.figure()
plot_confusion_matrix(confusionMatrix,normalize=True, classes=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
# print(np.max(resultsTest,axis=1))


In [ ]:
dictionary = {'acc': acc, 'f1': F1, 'prec': prec, 'mean_test_score': means, 'std_test_score': stds, 
              'confMatrix': confusionMatrix, 'bestParams': gridsearchcv.best_params_, 'testData': xMatrixTestExtended,
             'testLabels': y_testExtended}

sio.savemat('results_all_classes.mat', dictionary)

pickle.dump( gridsearchcv, open( "model_all_classes.p", "wb" ) )